In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = 'D:/datasets/dog_vs_cat/train/train'
TEST_DIR = 'D:/datasets/dog_vs_cat/test/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '6conv-basic-video')

In [2]:
def label_img(img):
    # dog.93.png > dog
    word_label = img.split('.')[-3]
    if word_label == 'cat': return np.array([1,0], dtype=float)
    elif word_label == 'dog': return np.array([0,1], dtype=float)

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img, dtype=float), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [81]:
# create_train_data()

In [5]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img, dtype=float), img_num])
    np.save('test_data.npy', testing_data)
    return testing_data

In [6]:
# train_data = create_train_data()
train_data = np.load('train_data.npy', allow_pickle=True)

In [14]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], dtype=float, name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


In [15]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('mpdel loaded!')

In [16]:
train = train_data[:-500]
test = train_data[-500:]

In [17]:
X = np.array([i[0] for i in train], dtype=float).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = np.array([i[1] for i in train], dtype=float)

test_x = np.array([i[0] for i in test], dtype=float).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = np.array([i[1] for i in test], dtype=float)

In [19]:
model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 4978  | total loss: 0.33770 | time: 9.170s
| Adam | epoch: 013 | loss: 0.33770 - acc: 0.8508 -- iter: 24448/24500
Training Step: 4979  | total loss: 0.33808 | time: 10.250s
| Adam | epoch: 013 | loss: 0.33808 - acc: 0.8532 | val_loss: 0.48359 - val_acc: 0.7880 -- iter: 24500/24500
--


In [ ]:
# tensorboard --logdir=C:\Users\ghdak\Desktop\Data_science\MUSIC\dogs_vs_cat\log  (anaconda prompt)

In [20]:
model.save(MODEL_NAME)

INFO:tensorflow:c:\Users\ghdak\Desktop\Data_science\MUSIC\dogs_vs_cat\dogsvscats-0.001-2conv-basic-video.model is not in all_model_checkpoint_paths. Manually adding it.


In [61]:
import matplotlib.pyplot as plt

In [84]:
def predict(example):
    if model.predict(example)[0][0] >  model.predict(example)[0][1]:
        return 'this is cat', model.predict(example)
    else:
        return 'this is dog', model.predict(example)
    

In [90]:
img_ex1 = cv2.resize(cv2.imread('aaa.jpg', cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
example1 = np.array(img_ex1, dtype=float).reshape((1, IMG_SIZE, IMG_SIZE, 1))
predict(example1)


('this is cat', array([[0.9887848 , 0.01121521]], dtype=float32))

In [ ]:
#to tmp

In [38]:
'''
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img, dtype=float), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data
'''

"\ndef create_train_data():\n    training_data = []\n    for img in tqdm(os.listdir(TRAIN_DIR)):\n        label = label_img(img)\n        path = os.path.join(TRAIN_DIR,img)\n        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))\n        training_data.append([np.array(img, dtype=float), np.array(label)])\n    shuffle(training_data)\n    np.save('train_data.npy', training_data)\n    return training_data\n"

In [ ]:
tflearn.DNN.predict